This is the template for the image recognition exercise. <Br>
Some **general instructions**, read these carefully:
 - The final assignment is returned as a clear and understandable *report*
    - define shortly the concepts and explain the phases you use
    - use the Markdown feature of the notebook for larger explanations
 - return your output as a *working* Jupyter notebook
 - name your file as Exercise_MLPR2023_Partx_uuid.jpynb
    - use the uuid code determined below
    - use this same code for each part of the assignment
 - write easily readable code with comments     
     - if you exploit code from web, provide a reference
 - it is ok to discuss with a friend about the assignment. But it is not ok to copy someone's work. Everyone should submit their own implementation
     - in case of identical submissions, both submissions are failed 

**Deadlines:**
- Part 1: Mon 6.2 at 23:59**
- Part 2: Mon 20.2 at 23:59**
- Part 3: Mon 6.3 at 23:59**

**No extensions for the deadlines** <br>
- after each deadline, example results are given, and it is not possible to submit anymore

**If you encounter problems, Google first and if you can’t find an answer, ask for help**
- Moodle area for questions
- pekavir@utu.fi
- teacher available for questions on Mondays 30.1, 13.2 (after lecture) and Thursday 2.3 (at lecture)

**Grading**

The exercise covers a part of the grading in this course. The course exam has 5 questions, 6 points of each. Exercise gives 6 points, i.e. the total score is 36 points.

From the template below, you can see how many exercise points can be acquired from each task. Exam points are given according to the table below: <br>
<br>
7 exercise points: 1 exam point <br>
8 exercise points: 2 exam points <br>
9 exercise points: 3 exam points <br>
10 exercise points: 4 exam points <br>
11 exercise points: 5 exam points <br>
12 exercise points: 6 exam points <br>
<br>
To pass the exercise, you need at least 7 exercise points, and at least 1 exercise point from each Part.
    
Each student will grade one submission from a peer and their own submission. After each Part deadline, example results are given. Study them carefully and perform the grading according to the given instructions. Mean value from the peer grading and self-grading is used for the final points. 

In [1]:
import uuid
# Run this cell only once and save the code. Use the same id code for each Part.
# Printing random id using uuid1()
print ("The id code is: ",end="")
print (uuid.uuid1())

The id code is: 41a9ddd6-bc55-11ed-807e-8eab187be911


# Introduction (1 p)

Write an introductory chapter for your report
<br>
- Explain what is the purpose of this task?
- Describe, what kind of data were used? Where did it originate? Give correct reference.
- Which methods did you use?
- Describe shortly the results

# Part 2

Data exploration and model selection

# Part 3

## Performance estimation (2 p)

Use the previously gathered data (again, use the standardized features). <br>
Estimate the performance of each model using nested cross validation. Use 10-fold cross validation for outer and <br>
5-fold repeated cross validation with 3 repetitions for inner loop.  <br> 
Select the best model in the inner loop using the hyperparameter combinations and ranges defined in the Part 2. <br>
For each model, calculate the accuracy and the confusion matrix. <br> 
Which hyperparameter/hyperparameter combination is most often chosen as the best one for each classifier? 

## Discussion (2 p)

Discuss you results

- Which model performs the best? Why?
- Ponder the limitations and generalization of the models. How well will the classifiers perform for data outside this data set?
- Compare your results with the original article. Are they comparable?
- Ponder applications for these type of models (classifying rice or other plant species), who could benefit from them? Ponder also what would be interesting to study more on this area?
- What did you learn? What was difficult? Could you improve your own working process in some way?

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from statistics import mean
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import KFold

In [3]:
df=pd.read_csv('notebooks/training_data/rice_feature_data.csv')

del df['Unnamed: 0']
del df['class_int']
df

,mean_b,var_b,skew_b,kurt_b,entr_b,mean_g,var_g,skew_g,kurt_g,entr_g,...,skew_r,kurt_r,entr_r,major_axis_length,minor_axis_length,area,perimeter,roundness,aspect_ratio,class
0,199.057107,4370.334017,-2.526412,4.854400,5.725611,188.949223,3968.899444,-2.489574,4.737988,5.789305,...,0.460606,-0.030300,5.751699,139.177719,67.960999,7418.0,362.818322,0.708139,2.047906,Arb
1,193.564744,3838.877309,-2.632029,5.449509,5.512506,189.985695,3734.693284,-2.587030,5.290814,5.620109,...,0.124742,0.246564,5.604786,133.097137,67.545555,6857.5,351.747255,0.696489,1.970480,Arb
2,211.430105,4253.600921,-2.799676,6.282459,5.609050,207.637314,4127.164001,-2.769280,6.175553,5.723262,...,-0.173684,-0.535170,5.691047,140.141525,70.338005,7423.0,364.190906,0.703285,1.992401,Arb
3,192.371917,4374.677145,-2.433318,4.327595,5.582211,187.144271,4157.929194,-2.413273,4.268942,5.599329,...,0.570922,-0.191927,5.599732,126.872337,61.993542,6096.5,327.261972,0.715318,2.046541,Arb
4,198.096162,3729.119794,-2.798684,6.272908,5.492145,194.080341,3612.771094,-2.754433,6.107209,5.586000,...,0.241334,0.682523,5.559394,135.600327,70.952782,7421.0,359.019333,0.723497,1.911135,Arb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,199.174473,5891.909855,-2.187297,2.850294,4.585445,190.771941,5412.518242,-2.181198,2.836350,4.644147,...,1.368763,1.081424,4.617680,148.539764,43.357121,4879.5,342.717816,0.522050,3.425960,Jas
296,187.794485,4556.494959,-2.373298,3.791280,4.976055,181.282682,4253.125209,-2.365083,3.770836,4.922925,...,1.605892,1.646148,4.906881,154.202957,45.328251,5181.5,344.717816,0.547946,3.401917,Jas
297,199.366926,5506.401941,-2.289285,3.319354,4.704367,188.118482,4914.260086,-2.278159,3.292082,4.820746,...,1.479758,1.191782,4.789066,151.572372,46.113976,5291.0,347.906636,0.549316,3.286908,Jas
298,188.160995,5116.081971,-2.218283,3.018641,4.837608,177.044129,4533.145771,-2.214500,3.009809,4.777169,...,1.752831,1.857270,4.884398,156.195450,46.311100,5504.0,359.563489,0.534979,3.372743,Jas


In [4]:
feats = ['mean_b', 'var_b', 'skew_b', 'kurt_b', 'entr_b', 'mean_g', 'var_g',
       'skew_g', 'kurt_g', 'entr_g', 'mean_r', 'var_r', 'skew_r', 'kurt_r',
       'entr_r', 'major_axis_length', 'minor_axis_length', 'area', 'perimeter',
       'roundness', 'aspect_ratio']

# Z-Score using pandas
for feat in feats:
    df['{}_Z'.format(feat)] = (df[feat] - df[feat].mean()) / df[feat].std()
    
feats_Z = [feat + '_Z' for feat in feats]

In [5]:
y = df['class'].values
X = df[feats_Z].values

In [6]:
warnings.filterwarnings('ignore')

k_range=range(1,30)
cv_inner = RepeatedKFold(n_splits=5, n_repeats=3, random_state=50)

# knn
knn = KNeighborsClassifier()
parameters={'n_neighbors': k_range}
gscv_knn = GridSearchCV(knn, parameters, cv=cv_inner, return_train_score=False)
gscv_knn.fit(X,y)
gscv_knn.best_params_['n_neighbors']

3

In [7]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X, y)

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)

scores = cross_val_score(knn, X, y, cv=cv_outer)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), np.std(scores)))

Accuracy: 0.990 (0.021)


In [8]:
y_pred=knn.predict(X)
metrics.confusion_matrix(y, y_pred)

array([[99,  1,  0],
       [ 0, 99,  1],
       [ 0,  1, 99]])

In [9]:
#random forest

max_depths = [2, 4, 6, 8, 10, 12]
max_features = [2, 3, 4, 5, 6, 7, 8]
parameters={'max_depth': max_depths,
           'max_features': max_features}

clf = RandomForestClassifier(random_state=20)
kf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=5)

gscv_rf = GridSearchCV(clf, parameters, cv=kf, return_train_score=False, n_jobs=-1)
gscv_rf.fit(X,y)
print('Best hyperparameter combination:')
print('Max_depth:', gscv_rf.best_params_['max_depth'])
print('Max_features:', gscv_rf.best_params_['max_features'])

Best hyperparameter combination:
Max_depth: 6
Max_features: 5


In [10]:
rf = RandomForestClassifier(max_depth=6, max_features=5)
rf.fit(X, y)

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)

scores = cross_val_score(rf, X, y, cv=cv_outer)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), np.std(scores)))

Accuracy: 0.987 (0.022)


In [11]:
y_pred=rf.predict(X)
metrics.confusion_matrix(y, y_pred)

array([[100,   0,   0],
       [  0, 100,   0],
       [  0,   0, 100]])

In [12]:
#mlp

mlp = MLPClassifier(max_iter=500, early_stopping=True, random_state=20)

hidden_layer_sizes = [(i,) for i in range(3,22)]
activation = ['logistic', 'relu']
solver = ['sgd', 'adam']
val_fr = [0.1, 0.5]

parameter_space = {
    'hidden_layer_sizes': hidden_layer_sizes,
    'activation': activation,
    'solver': solver,
    'validation_fraction': val_fr
}

kf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=5)
# select the best hyperparameter set
gscv = GridSearchCV(mlp, parameter_space, cv=cv_inner)
gscv.fit(X, y)
print('Best hyperparameter combination:')
gscv.best_params_

Best hyperparameter combination:


{'activation': 'relu',
 'hidden_layer_sizes': (14,),
 'solver': 'adam',
 'validation_fraction': 0.5}

In [13]:
clf = MLPClassifier(solver='adam', activation = 'relu',
                    hidden_layer_sizes=(14,), validation_fraction=0.5)

clf.fit(X,y)

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
scores = cross_val_score(clf, X, y, cv=cv_outer)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), np.std(scores)))

Accuracy: 0.983 (0.022)


In [14]:
y_pred=clf.predict(X)
metrics.confusion_matrix(y, y_pred)

array([[ 99,   1,   0],
       [  0,  99,   1],
       [  0,   0, 100]])

In [15]:
# The Random Forest model performed the best.
# One reason why Random Forest can perform well in image processing is that it can handle high-dimensional input data, 
# such as images, which have a large number of features. 
# Random Forest can effectively reduce the dimensionality of the input data by selecting a subset of features 
# at each node of the decision tree, which helps to reduce the risk of overfitting and 
# improve the model's generalization performance.

# Another reason why Random Forest can perform well in image processing is that it is able to 
# capture non-linear relationships between the input features and the output classes. 
# This is important in image classification and segmentation tasks, where the relationship 
# between the input features and the output classes can be highly complex and non-linear.

# Finally, Random Forest is also able to handle missing data and outliers, 
# which can be common in image processing tasks. Random Forest can use the information from the other decision trees 
# in the ensemble to fill in missing data and reduce the impact of outliers, 
# which can help to improve the model's robustness and performance.

#Machine learning models have limitations and generalization issues that can affect their performance 
#and applicability in real-world scenarios. Some of these limitations and generalization issues include:
#Overfitting: Overfitting occurs when the model is too complex and captures noise 
#in the training data rather than the underlying patterns. This can lead to poor performance on new, unseen data.
#Underfitting: Underfitting occurs when the model is too simple and does not capture the underlying patterns 
#in the data. This can lead to poor performance on both the training and test data.

# The image technology can be used to classify rice in the rice industry, 
# or it can be used as a technology to quickly identify animal species with animal data sets. 
# It would also be interesting to implement real-time image processing by combining it with computer vision 
# in this field.

# Part 1 was a little difficult. 
# However, I felt that the difficulty level became easier as I moved on to part2. 
# However, I was a little lost because it was my first time encountering hyperparameter tuning. 
# Still, as I continued to try and learn about hyperparameter tuning, I was able to easily fulfill the task in part 3.
# I think that if you continue to practice using actual data in the future, your skills will be better than now.
